Analyzing Solutions
===================

While every mode matching that achieves perfect overlap should be equivalent in theory, in practice some solutions are much easier to implement and align than others. If a lot of degrees of freedom are offered to the solver, there may be many possible solutions, and it can hard to sift through them all to find the best one. To help with this, Beam Corset calculates a variety of metrics and two kinds of analysis plots to help you find the most practical solution.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from corset import Beam, ThinLens, ShiftingRange, mode_match

In [ ]:
# setup beam with some uncertainty for later analysis
initial_beam = Beam.from_gauss(0, waist=500e-6, wavelength=1064e-9, cov=np.diag([1e-2, 10e-6])**2)
desired_beam = Beam.from_gauss(focus=1.0, waist=100e-6, wavelength=initial_beam.wavelength)
lenses = [ThinLens(f, 10e-3, 10e-3) for f in [100e-3, 150e-3, 200e-3]]
ranges = [ShiftingRange(left=0.0, right=0.8)]
solutions = mode_match(initial_beam, desired_beam, ranges, lenses, max_elements=3)
print(f"Found {len(solutions)} solutions.")

We give the solver three different kinds of lenses to work with and allow up to three elements per mode matching to get a larger number of possible solutions to analyze.

Quantitative Analysis
---------------------

To analyze a solution quantitatively, we can look at its :attr:`~corset.solver.ModeMatchingSolution.analysis` attribute which is a lazily computed :class:`~corset.analysis.ModeMatchingAnalysis` instance. The analysis object computes a variety of metrics like sensitives of the individual degrees of freedom and how they affect the final beam's waist position and radius.

For example, one very good predictor of how easy a solution is to implement are the :attr:`~corset.analysis.ModeMatchingAnalysis.couplings` between the different degrees of freedom. It essentially describes how easily we can optimize the mode matching by only moving one element at a time without having to move another element with it to keep the overlap high. A coupling close to $1$ or $-1$ means that we will see a significant drop in overlap if we don't move these two elements together with a precise ratio, a coupling close to $0$ means that the two elements can mostly be adjusted independently of each other.

In [ ]:
print(solutions[0].analysis.sensitivities) # sensitivity matrix
print(solutions[0].analysis.couplings) # coupling matrix
print(solutions[0].analysis.grad_focus) # gradient of focus positions w.r.t. degrees of freedom
# ...

We can get a quick summary of all these quantities and some additional information by calling its :meth:`~corset.analysis.ModeMatchingAnalysis.summary` method.

In [ ]:
solutions[0].analysis.summary() # all analyses and some more information in a dictionary

The sensitivity matrix is one of the most important metrics, that many other quantities are derived from. It is proportional to the Hessian of the mode overlap with respect to the degrees of freedom such that the positive loss in overlap around the optimum is approximately given by:

$$
\Delta o \approx \Delta \mathbf{x}^T \mathbf{S} \Delta \mathbf{x}
$$

where $\mathbf{S}$ is the sensitivity matrix and $\Delta \mathbf{x} = [\Delta x_0, \Delta x_1, \ldots]^T$ is the vector of displacements in the degrees of freedom.

By default, the sensitivity is specified in units of $\%/\text{cm}^2$ where typical values are around unity. This makes it very easy to estimate how much overlap is lost for a given displacement. For example if the sensitivity is $1.5\,\%/\text{cm}^2$ and we displace the element by $1\text{ cm}$, we can square it to get a loss of approximately $1.5\%$ in overlap. If we displace it by $2\text{ cm}$ the loss will be about $6\%$.

The coupling matrix $\mathbf{R}$ is the sensitivity matrix normalized by its diagonal elements in the same way, that the correlation matrix is derived from the covariance matrix:

$$
r_{ij} = \frac{s_{ij}}{\sqrt{s_{ii} s_{jj}}}
$$

You can find a list of all available metrics, their meanings, and how they are calculated in the :class:`~corset.analysis.ModeMatchingAnalysis` class documentation.

Instead of calling the summary method on each solution individually, we can also simply display the :class:`~corset.solver.SolutionList` returned by the solver, which shows a table of all summaries at once.

In [ ]:
solutions

The :class:`~corset.solver.SolutionList` also provides some other convenience functions to filter and sort the solutions based on the analysis results. For example, we can filter the solutions by their number of elements using the lists :meth:`~corset.solver.SolutionList.query` method, and the sort them by the coupling of the least coupled pair of freedom (:attr:`~corset.solver.SolutionList.min_coupling`) with the :meth:`~corset.solver.SolutionList.sort_values` method:

In [ ]:
solutions.query("num_elements == 2").sort_values("min_coupling") # ascending by default

These methods have the same names as :meth:`pandas.DataFrame.query` and :meth:`pandas.DataFrame.sort_values` because they are implemented by running these methods on the preview :class:`~pandas.DataFrame` and returning a new :class:`~corset.solver.SolutionList` with the filtered/sorted solutions. This means they also take the same inputs as these methods.

Analysis Plots
--------------

Let's take a look at the most and least coupled solution to explain the features of the reachability and analysis plots in detail.

In [ ]:
solutions_by_coupling = solutions.sort_values("min_coupling")

### Reachability Plot

The reachability analysis can be plotted by using the :func:`~solver.ModeMatchingSolution.plot_reachability` member function of the respective solution. We can pass the optional `ax` argument to plot both of them in the same figure.

In [ ]:
fig, (axl, axr) = plt.subplots(1, 2, figsize=(12, 5))
solutions_by_coupling[0].plot_reachability(ax=axl)
solutions_by_coupling[-1].plot_reachability(ax=axr);

The reachability plot shows which waist positions and radii can be _reached_ by varying the degrees of freedom in a certain range around the calculated optimum represented by a black dot. Assuming perfect alignment, the dashed ellipse represents the confidence region of the waist position and radius, propagated from the fit uncertainty. In another sense, we can also interpret this region as the deviation in waist position and radius that we will likely need to compensate for because of differences in the input beam to our fitted input beam parameters.

The resulting focus and waist positions are evaluated on a grid spanned by all combinations of displacements in the individual degrees of freedom. The points are then connected by colored lines that indicate the degree of freedom that changed between the two points. The direction that corresponds to a positive change in the coordinate is indicated by an arrow starting at the negative most point of all degrees of freedom.

In addition to differentiating waist position from radius, the axis labels also indicate the sensitivity or gradient of the waist position/radius with respect to changes in the degrees of freedom at the optimum.

Finally, the background is a contour plot of the mode overlap for the different final beam positions and radii.

With all this information, we can infer many characteristics of the solution like:
- the adjustments in waist position and radius that achieve with reasonable adjustments of the degrees of freedom,
- and how these adjustments affect the mode overlap,
- this also tells us how (in)sensitive the final beam is to the individual degrees of freedom,
- which degree of freedom affects the final beam's waist position and radius in which way or ratio,
- if the actions of the different degrees are sufficiently independent to each other, i.e., orthogonal,
- how likely we are to be able to compensate for mode mismatch due to deviations from the fitted input beam parameters.

Comparing the two solutions, we can see that in the least coupled solution (left), the three degrees of freedom are all fairly independent of each other with $x_0$ (blue) and $x_2$ (green) almost orthogonal to each other. We can also see, that we will most likely be able to compensate for our fit uncertainties by performing reasonable adjustments to the lens positions that are likely within the range of a translation stage. The strongly coupled (right) solution is pretty much the opposite of this, $x_1$ (orange) and $x_2$ (green) are almost perfectly aligned while $x_0$ (blue) only makes a small angle with them. The result of these couplings is that we require very large and combined movements to achieve a significant correction in the waist radius causing us likely be unable to compensate for the fit uncertainties.

### Sensitivity Plot

As the name would imply, we can create a sensitivity plot using the :func:`~solver.ModeMatchingSolution.plot_sensitivity` member function of the respective solution.

In [ ]:
fig, (axl, axr) = plt.subplots(1, 2, figsize=(12, 5))
solutions_by_coupling[0].plot_sensitivity(ax=axl)
solutions_by_coupling[-1].plot_sensitivity(ax=axr);

The sensitivity plot, as the name implies, shows how sensitive the mode overlap is to changes in the degrees of freedom or combinations thereof. The plot is a contour plot of the mode overlap as a function of displacements in the two least coupled degrees of freedom. If there are more than two degrees of freedom, the most sensitive remaining degree of freedom is varied in both directions and the resulting contour plots are visualized using different colors. If there are only two degrees of freedom the counter plot is plotted as a contour fill with the same color map as the reachability plot (see next subsection for an example of this). For three or more degrees of freedom only labeled contour lines are drawn to keep the plot readable.

In addition to indicating the respective degrees of freedom, the axis labels also indicate the sensitivity of the respective degree of freedom and the title shows the coupling between the two primary degrees of freedom. And as with the reachability plot, the black dashed ellipse indicates the confidence region propagated from the fit uncertainty. In this case it should be interpreted as the corrections that might be necessary due to deviations from the fitted input beam parameters.

The range for the displacements is automatically chosen so that the 98% mode overlap contour is contained within the plot area. The computation is based on a heuristic so it is not guaranteed that the contour line is always fully visible.

Since mode matching only requires two degrees of freedom, solutions with more than two degrees of freedom are underdetermined. In the mathematical sense, this means that the sensitivity matrix is no longer positive definite, instead it has one zero (or close to zero) eigenvalue per extra degree of freedom. Practically this means, that deviations in one lens can be fully compensated by adjustments in the other lenses without any loss in overlap. This is why the different slices of the auxiliary degree of freedom have mostly the same contour lines. The range of the auxiliary degree of freedom is determined so that the minima within the slices are just within the plotting area. This is also determined heuristically, so the minima are not always guaranteed to be visible.

Comparing the two sensitivity plots, we can see that the inner contour lines of the least coupled (left) solution are fairly circular while the contour lines of the strongly coupled (right) solution are extremely elliptical with non axis aligned major axis. This comparison is also reflected in the coupling values shown in the titles of the plots. This is in some sense the dual representation of the reachability plot, to go from the reachability plot, we distort the plot so that the two primary degrees of freedom are orthogonal and axis aligned. If they already were close to orthogonal the transformation is mostly rotating and scaling. However, if basis vectors were almost pointing in the same direction, the distortion needed to get them orthogonal is much larger which leads to these strongly elliptical contour lines.

As we get farther away from the optimum, the quadratic approximation that a lot of the analysis is based becomes less accurate, and we can see the contour lines distort from their elliptical shape to much more complex forms.

Another important aspect that is somewhat hidden by the automatic range selection is the sensitivity values shown in the axis labels. Due to the strong coupling, the sensitivities of the respective degrees of freedom is three to ten times larger in the strongly coupled solution compared to the least coupled solution.

### Conclusion

While it is possible to create the reachability and analysis plots by themselves, you will usually invoke them using the :meth:`~corset.solver.ModeMatchingSolution.plot_all` function which also shows an overview of the mode matching setup in addition to the two analysis plots.

In [ ]:
solutions_by_coupling[0].plot_all();

This function is also responsible for creating the PNG representation for solution objects that is shown when a :class:`~corset.solver.ModeMatchingSolution` is the last expression of a cell or when it is explicitly passed to the :func:`~IPython.display.display` function. We can use this to easily get an overview of multiple solutions to compare them at a glance.

In [ ]:
display(*solutions_by_coupling[:3]) # unpack the slice into the display arguments
display(solutions_by_coupling[-1])